# To launch selected cell use `Ctrl + Enter`

# Requirements

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp

from manim import *

config.media_width = "75%"
config.verbosity = "WARNING"

def get_continuous_y(t, time_range, y_values):
    
    t_nearest = min(time_range, key=lambda x: abs(x-t))
    i = time_range.index(t_nearest)

    if t < t_nearest:
        i1, i2 = i-1, i
    elif t > t_nearest:
        i1, i2 = i, i+1
    else:
        return y_values[i]
        
    t1, t2, y1, y2 = time_range[i1], time_range[i2], y_values[i1], y_values[i2]

    return (t - t1) / (t2 - t1) * (y2 - y1) + y1

# Setting the system parameters

# $\theta''+\frac{b}{m}\theta'+\frac{l}{g}\sin{\theta}=0$ is our equation

An explanation of all the maths will be here later. Now you can change parameters and see where it leads. I don't recommend setting high angles because the phase portrait will just go out of frame, it's not done yet.

In [ ]:
b, m, g, l, T = 0.5, 1, 1, 1, 23 # T is simulation time
S0 = (0, np.pi/2 + np.pi/6) # initial conditions

# Solution calculation

In [ ]:
def dSdt(t,S):
    x1, x2 = S
    return [x2, -b/m*x2-g/l*np.sin(x1)]

t = np.linspace(0, T, 10001)
sol = solve_ivp(dSdt, t_span=(0,max(t)), y0=S0, t_eval=t)#, method = 'DOP853', rtol=1e-10, atol=1e-13)

time_range = tuple(sol.t)
y_values_0 = sol.y[0]
y_values_1 = sol.y[1]

# Animation (it takes some time, be patient)

In [ ]:
%%manim -ql Phase_Space

Pendulum_Point = 4*LEFT + 2*UP
Phase_Point = 2.5*RIGHT
        
class Phase_Space(Scene):
    
    def phase(self, t):
        return Phase_Point + 7.5/4*np.array((get_continuous_y(t, time_range, y_values_0), get_continuous_y(t, time_range, y_values_1), 0))
    
    def pend(self, t):
        return Pendulum_Point + 3*np.array((np.sin(get_continuous_y(t, time_range, y_values_1)), -np.cos(get_continuous_y(t, time_range, y_values_1)), 0))
    
    def construct(self):
        
        fps = 15
        
        ### phase space
        
        path = ParametricFunction(self.phase, t_range = np.array([0, T]), fill_opacity=0).set_color(RED)
        point = Dot(self.phase(0), radius=0.1).set_color(YELLOW)
        
        self.add(path)
        
        ### Axes
        
        self.wait()
        
        axes = Axes(
            x_range=[-2, 2],
            y_range=[-2, 2],
            x_length=7.5, 
            y_length=7.5,            
            axis_config={                
                "include_tip": True,                
                #"color": GREY,
                "stroke_width": 2,
                "font_size": 24, 
                "tick_size": 0.07,                
                "longer_tick_multiple": 1.5,
                "line_to_number_buff": 0.15,  
                "decimal_number_config": {
                    #"color": ORANGE,
                    "num_decimal_places": 0
                    },             
                },
            x_axis_config={
                "tip_width": 0.15,
                "tip_height": 0.15,
                "numbers_to_include": np.arange(-2, 3, 1),
                "numbers_with_elongated_ticks": [-1, 1],
                },             
            y_axis_config={
                "tip_width": 0.15,
                "tip_height": 0.15,                
                "numbers_to_include": np.arange(-2, 3, 1),
                "tick_size": 0.08,
                "font_size": 25, 
                "numbers_with_elongated_ticks": [-1, 1],                
                }
        ).move_to(Phase_Point)
        x_lab = axes.get_x_axis_label("\\theta'", direction=UP, buff=0.2)
        y_lab = axes.get_y_axis_label("\\theta", direction=RIGHT, buff=0.2)
        labels = VGroup(x_lab.scale(0.9), y_lab.scale(0.9))

        self.play(Write(axes, run_time=5), lag_ratio=0.2)
        self.play(Write(labels))        
        self.wait()
        
        ###
        
        self.play(FadeIn(point))
        
        ### pendulum
        
        mass = Dot(self.pend(0), radius=0.3).set_color(BLUE)
        dashed_line = DashedLine(ORIGIN + Pendulum_Point, 3*DOWN + Pendulum_Point).set_color(GRAY)
        line = always_redraw(lambda: Line(ORIGIN + Pendulum_Point, mass.get_center()))
        trace = TracedPath(mass.get_center, dissipating_time=0.5, stroke_opacity=[0, 1], stroke_color='#FF8C00', stroke_width=10)
        
        self.play(
            AnimationGroup(
                Create(dashed_line),
                Create(line),
                DrawBorderThenFill(mass),
                Create(trace),
                lag_ratio=0.1
            )
        )
        
        ### render
        
        self.wait()
        
        for i in np.linspace(0, T, T*fps+1):
            self.play(
                AnimationGroup(
                    ApplyMethod(point.move_to, self.phase(i)),
                    ApplyMethod(mass.move_to, self.pend(i)),
                    run_time=1/fps,
                    lag_ratio=0
                )
            )
        
        self.wait()

Сhange the key -ql (low quality 480p 15fps) to -qm (medium quality 720p 30fps) or even -qh (high quality 1080p 60fps) for better quality and be sure to change the `fps` value in the cell above.